In [16]:
from nba_api.stats.endpoints import leaguegamelog
import time
def get_all_games(start_year=1946, end_year=2023):
  results = []
  for year in range(start_year, end_year):
    season = '{}-{}'.format(year, str(year + 1)[-2:])
    res = leaguegamelog.LeagueGameLog(counter=10000, season=season)
    res_frame = res.get_data_frames()[0]
    results.append(res_frame)
    time.sleep(1)
  return results
    
results = get_all_games(2022, 2024)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,240,45,...,37,48,31,11,4,18,23,123,14,1
1,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,240,40,...,39,48,23,12,4,22,18,109,-14,1
2,22022,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,46,...,30,36,24,8,3,11,24,126,9,1
3,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,240,40,...,27,31,16,8,3,14,25,117,-9,1
4,22022,1610612756,PHX,Phoenix Suns,0022200013,2022-10-19,PHX vs. DAL,W,240,40,...,32,40,25,4,5,12,29,107,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22022,1610612742,DAL,Dallas Mavericks,0022201224,2023-04-09,DAL vs. SAS,L,240,46,...,38,46,28,4,3,4,21,117,-21,1
2456,22022,1610612759,SAS,San Antonio Spurs,0022201224,2023-04-09,SAS @ DAL,W,240,51,...,56,69,31,2,3,12,13,138,21,1
2457,22022,1610612750,MIN,Minnesota Timberwolves,0022201225,2023-04-09,MIN vs. NOP,W,240,39,...,34,42,25,7,8,17,22,113,5,1
2458,22022,1610612740,NOP,New Orleans Pelicans,0022201225,2023-04-09,NOP @ MIN,L,240,40,...,38,49,18,13,3,15,24,108,-5,1


In [18]:
results[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          2460 non-null   object 
 1   TEAM_ID            2460 non-null   int64  
 2   TEAM_ABBREVIATION  2460 non-null   object 
 3   TEAM_NAME          2460 non-null   object 
 4   GAME_ID            2460 non-null   object 
 5   GAME_DATE          2460 non-null   object 
 6   MATCHUP            2460 non-null   object 
 7   WL                 2460 non-null   object 
 8   MIN                2460 non-null   int64  
 9   FGM                2460 non-null   int64  
 10  FGA                2460 non-null   int64  
 11  FG_PCT             2460 non-null   float64
 12  FG3M               2460 non-null   int64  
 13  FG3A               2460 non-null   int64  
 14  FG3_PCT            2460 non-null   float64
 15  FTM                2460 non-null   int64  
 16  FTA                2460 